# serve

> A parallel ipynb processor (experimental)
- order: 17

In [ ]:
#| default_exp serve

In [ ]:
#|export
import ast,subprocess,threading,sys,traceback
from shutil import rmtree,copy2
from contextlib import contextmanager

from fastcore.utils import *
from fastcore.parallel import parallel
from fastcore.script import call_parse
from fastcore.meta import delegates

from nbdev.config import get_config
from nbdev.doclinks import nbglob_cli,nbglob
from nbdev.processors import FilterDefaults
from nbdev import serve_drv

In [ ]:
#|hide
__file__ = '16_serve.ipynb'

In [ ]:
#|export
def _is_qpy(path:Path):
    "Is `path` a py script starting with frontmatter?"
    path = Path(path)
    if not path.suffix=='.py': return
    p = ast.parse(path.read_text())
#     try: p = ast.parse(path.read_text())
#     except: return
    if not p.body: return
    a = p.body[0]
    if isinstance(a, ast.Expr) and isinstance(a.value, ast.Constant):
        v = a.value.value.strip()
        vl = v.splitlines()
        if vl[0]=='---' and vl[-1]=='---': return v

def _exec_py(fname):
    "Exec a python script and warn on error"
    try: subprocess.check_output('python ' + fname, shell=True)
    except subprocess.CalledProcessError as cpe: warn(str(cpe))

In [ ]:
#|export
def _proc_file(s, cache, path, mtime=None):
    d = cache/s.relative_to(path)
    if s.suffix=='.py': d = d.with_suffix('.qmd')
    if d.exists():
        dtime = d.stat().st_mtime
        if mtime: dtime = max(dtime, mtime)
        if s.stat().st_mtime<=dtime: return

    d.parent.mkdir(parents=True, exist_ok=True)
    if s.suffix=='.ipynb': return s,d,FilterDefaults
    md = _is_qpy(s)
    if md: return s,d,md
    else: copy2(s,d)

In [ ]:
#|export
@call_parse
@delegates(nbglob_cli)
def proc_nbs(
    path:str='', # Path to notebooks
    n_workers:int=defaults.cpus,  # Number of workers
    force:bool=False,  # Ignore cache and build all
    file_glob:str='', # Only include files matching glob
    **kwargs):
    "Process notebooks in `path` for docs rendering"
    from multiprocessing.forkserver import set_forkserver_preload
    set_forkserver_preload(['io', 'contextlib', 'execnb.nbio'])

    cfg = get_config()
    cache = cfg.config_path/'_proc'
    path = Path(path or cfg.nbs_path)
    files = nbglob(path, func=Path, file_glob=file_glob, **kwargs)
    if (path/'_quarto.yml').exists(): files.append(path/'_quarto.yml')

    # If settings.ini or filter script newer than cache folder modified, delete cache
    chk_mtime = max(cfg.config_file.stat().st_mtime, Path(__file__).stat().st_mtime)
    cache.mkdir(parents=True, exist_ok=True)
    cache_mtime = cache.stat().st_mtime
    if force or (cache.exists and cache_mtime<chk_mtime): rmtree(cache)

    files = files.map(_proc_file, mtime=cache_mtime, cache=cache, path=path).filter()
    kw = {} if IN_NOTEBOOK else {'method':'forkserver' if os.name=='posix' else 'spawn'}
    parallel(serve_drv.main, files, n_workers=n_workers, pause=0.01, **kw)
    if cache.exists(): cache.touch()
    return cache

In [ ]:
#|hide
# proc_nbs.__wrapped__(n_workers=0)

In [ ]:
#|export
@contextmanager
def fs_watchdog(func, path, recursive:bool=True):
    "File system watchdog dispatching to `func`"
    from watchdog.observers import Observer
    from watchdog.events import FileSystemEventHandler
    class _ProcessHandler(FileSystemEventHandler): dispatch=func
    observer = Observer()
    observer.schedule(_ProcessHandler, path, recursive=True)
    observer.start()
    try: yield
    except KeyboardInterrupt: pass
    finally:
        observer.stop()
        observer.join()

In [ ]:
#|export
def preview_server(path:str='', xtra=None):
    cache = get_config().config_path/'_proc'
    path = Path(path or get_config().nbs_path)
    def _f(e):
        src = Path(e.src_path)
        if src.is_file() and not any(o[0]=='.' for o in src.parts):
            res = _proc_file(src, cache, path)
            if res:
                try: serve_drv.main(res)
                except: traceback.print_exc()

    os.chdir(cache)
    xtra = xtra or []
    with fs_watchdog(_f, path): subprocess.run(['quarto','preview']+xtra)

In [ ]:
# preview_server()

## Export -

In [ ]:
#|hide
import nbdev; nbdev.nbdev_export()